# **Advanced CUDA**

Add CUDA to path in Jupyter Notebook even though nvcc compiler is detected in terminal, as it is not directly detected by ipykernel.

In [1]:
import os
os.environ["PATH"] += ":/usr/local/cuda/bin"

# Verify nvcc is now accessible
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Tue_Oct_29_23:50:19_PDT_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0


---
## **01 - Atomic Operations**

An atomic operation in CUDA is a type of operation that is performed in a way that ensures it is indivisible—that is, it cannot be interrupted or affected by other threads. When multiple threads attempt to modify a shared memory location, atomic operations ensure that these modifications are executed one at a time, avoiding race conditions.

For example, when multiple threads try to increment a shared counter, an atomic operation ensures that each increment happens sequentially, even if threads are running concurrently.

**Why Are Atomic Operations Necessary?**

In parallel programming, multiple threads often need to access or update shared data. Without synchronization mechanisms like atomic operations, the following issues can arise:
- Race Conditions: Multiple threads attempt to update the same variable simultaneously, leading to inconsistent results.
- Data Corruption: Intermediate results of one thread's operation can be overwritten by another thread.
- Incorrect Computation: Operations that depend on shared data (e.g., summation, counting) may produce wrong results due to simultaneous accesses.

Atomic operations prevent these problems by serializing access to the shared resource, ensuring that only one thread modifies the variable at a time.

**Common Atomic Operations in CUDA**

CUDA provides several atomic functions that operate on different data types and perform common operations:

- Arithmetic Operations:
    - atomicAdd: Adds a value to a shared variable.
    - atomicSub: Subtracts a value from a shared variable.
    - atomicExch: Replaces a value with a new one.

- Comparison and Logical Operations:
    - atomicMin: Updates the variable with the minimum of the current and provided value.
    - atomicMax: Updates the variable with the maximum of the current and provided value.
    - atomicCAS (Compare and Swap): Updates a variable only if it equals a specified value.

- Bitwise Operations:
    - atomicAnd: Performs a bitwise AND.
    - atomicOr: Performs a bitwise OR.
    - atomicXor: Performs a bitwise XOR.

**Adding array of 1024 1s**

In [2]:
!make SRC=./src/01_atomic_operations.cu run

nvcc -o ./src/01_atomic_operations ./src/01_atomic_operations.cu
././src/01_atomic_operations
Sum of array elements (normalSumKernel): 1
Sum of array elements (atomicSumKernel): 1024


**Code Explanation**

- Normal Sum (`normalSumKernel`):
Each thread reads from the input array and adds its value to the shared variable result.
Issue: Without atomic operations, multiple threads may update result simultaneously, leading to race conditions and an incorrect sum.

- Atomic Sum (`atomicSumKernel`):
Uses `atomicAdd` to safely add each thread’s contribution to result.
Solution: Ensures that only one thread updates result at a time, preventing race conditions and producing the correct sum.

- Result Comparison:
`normalSumKernel`: Results are incorrect because threads overwrite each other's updates.
`atomicSumKernel`: Produces the correct sum by using atomic operations to serialize updates.

**Why Atomic Operations Are Crucial in This Code?**

- The shared variable result is updated concurrently by multiple threads.
- Without atomicAdd, updates are not safe in parallel, leading to data corruption.
- `atomicAdd` ensures correctness but can slow performance due to thread serialization.

**Counter using Normal Addition and Atomic Addition**

In [3]:
!make SRC=./src/01_atomic_operations.cu clean

rm -f ./src/01_atomic_operations


---
## **02 - Events**

CUDA events are a mechanism in the CUDA API used to measure the time taken by operations on the GPU or to synchronize operations between different streams. CUDA events are lightweight and designed specifically for timing and synchronization tasks in GPU programming.
Necessity of CUDA Events

- Performance Measurement: CUDA events allow you to measure the execution time of GPU operations accurately.
- Synchronization: Events can synchronize streams or host-device operations without blocking the entire application.
- Granular Timing: They provide more precise control and insight compared to cudaDeviceSynchronize or host-based timers.

In [4]:
!make SRC=./src/02_events.cu run

nvcc -o ./src/02_events ./src/02_events.cu
././src/02_events
Kernel execution time: 0.902464 ms


**Create Events:**

`cudaEvent_t start, stop;`

`cudaEventCreate(&start);`

`cudaEventCreate(&stop);`

- `start` and `stop` are event handles.

**Record Events:**

`cudaEventRecord(start);`

- Start recording before the kernel execution.

**Synchronize Events:**

`cudaEventSynchronize(stop);`

- Ensures all operations before stop are completed.

**Calculate Elapsed Time:**

`cudaEventElapsedTime(&milliseconds, start, stop);`

- Computes time in milliseconds between the start and stop events.

**Destroy Events:**

`cudaEventDestroy(start);`

`cudaEventDestroy(stop);`

- Cleans up event resources.

In [5]:
!make SRC=./src/02_events.cu clean

rm -f ./src/02_events


---
## **03 - Streams**

**CUDA Streams: Enhancing GPU Parallelism**

CUDA streams are a powerful feature in NVIDIA's CUDA programming model that allow for concurrent execution of operations on the GPU. They provide an additional layer of parallelism beyond the traditional thread and block model, enabling more efficient utilization of GPU resources.
- Key Concepts

    - Definition: A CUDA stream is a sequence of operations that execute on the GPU in a specific order.
    - Purpose: Streams enable concurrent execution of kernels and memory transfers, improving overall performance2.
    - Default Stream: All CUDA operations occur in the default stream if not specified otherwise1.

- Stream Behavior

    - Ordering: Operations within a single stream are executed sequentially.
    - Concurrency: Different non-default streams can execute operations concurrently.
    - Default Stream Behavior: The default stream is blocking and synchronizes with all other streams.

<div style="text-align: center;">
  <img src="./images/cuda_streams.bmp" alt="CUDA Streams" width="800">
</div>

This image illustrates the performance difference between serial and concurrent CUDA stream execution.

- The top portion shows a Serial execution where operations happen sequentially:

    1. Memory copy from Host to Device (H2D)
    2. Kernel execution
    3. Memory copy from Device to Host (D2H)

- The bottom portion shows Concurrent execution using three streams:

    - Stream 1, 2, and 3 execute their operations (H2D, Kernel, D2H) in parallel
    - Operations within each stream remain sequential
    - Streams are staggered in time, allowing overlap of different operations

- The red dotted lines highlight the Performance improvement achieved through concurrent execution, showing how parallel streams complete the same workload in less time compared to serial execution. The green boxes represent memory transfers (H2D and D2H), while the blue boxes represent kernel executions. 

This visualization effectively demonstrates how CUDA streams can improve GPU utilization by overlapping computation and data transfer operations.

**Benefits of Using Streams**

- Improved GPU Utilization: Overlapping kernel execution with data transfers.
- Reduced Idle Time: Keeping the GPU busy with multiple concurrent operations.
- Enhanced Performance: Achieving higher throughput for certain workloads.

**Stream Create, Synchronize, and Destroy Syntax**

You can declare a stream with `cudaStream_t`, create it using `cudaStreamCreate(&stream)`. To synchronize a stream, use `cudaStreamSynchronize(stream)`, ensuring that all tasks in the stream finish before proceeding. Finally, free resources with `cudaStreamDestroy(stream)`.

The `cudaStreamSynchronize` function is a crucial synchronization tool that blocks the host thread until all previously queued operations in the specified stream complete their execution.

Usage Scenarios:
- Ensuring data consistency before host access.
- Coordinating multiple stream operations.
- Managing dependencies between CPU and GPU tasks.


**Syntax to launch kernel in a Stream**

`myKernel<<<gridSize, blockSize, sharedMem, stream>>>(parameters);`


- `gridSize`: Specifies the number of thread blocks in the grid.
- `blockSize`: Defines the number of threads in each block.
- `sharedMem`: Amount of shared memory to allocate per block (in bytes).
- `stream`: Specifies which CUDA stream will execute this kernel.

**Squaring number - Without CUDA Stream**

In [23]:
!make SRC=./src/03a_no_streams.cu run

nvcc -o ./src/03a_no_streams ./src/03a_no_streams.cu
././src/03a_no_streams
Execution time without streams: 1.1448 ms


**Squaring number - With CUDA Stream (Individually created streams)**

In [21]:
!make SRC=./src/03b_with_streams.cu run

nvcc -o ./src/03b_with_streams ./src/03b_with_streams.cu
././src/03b_with_streams
Execution time with streams: 2.86925 ms


**Squaring number - With CUDA Stream (Streams created in `for` loop)**

In [22]:
!make SRC=./src/03c_with_streams_for.cu run

nvcc -o ./src/03c_with_streams_for ./src/03c_with_streams_for.cu
././src/03c_with_streams_for
Execution time with streams: 2.4617 ms


The speedup occurs because:

- The workload is divided into 4 independent streams
- Memory transfers and kernel executions overlap across streams
- While one stream is executing its kernel, another stream can be performing memory transfers
- The GPU's hardware resources are utilized more efficiently through concurrent execution

This example demonstrates how CUDA streams can significantly improve performance by enabling parallel execution of operations that would otherwise need to wait for previous operations to complete in a serial implementation.

In [33]:
!make SRC=./src/03a_no_streams.cu clean
!make SRC=./src/03b_with_streams.cu clean
!make SRC=./src/03c_with_streams_for.cu clean

rm -f ./src/03a_no_streams


rm -f ./src/03b_with_streams
rm -f ./src/03c_with_streams_for


---
## **04 - Memory Coalescing**

<p align="center">
    <img src="./images/uncoalesced_memory.png" alt="Uncoalesced Memory Access" width="30%">
    <img src="./images/coalesced_memory.png" alt="Coalesced Memory Access" width="30%">
</p>

Memory coalescing is a crucial optimization technique in parallel computing, particularly for GPU architectures like CUDA. It refers to the process of merging multiple memory accesses into a single, larger access to improve data transfer efficiency. This technique is especially important for optimizing performance in systems with hierarchical memory models, where non-coalesced memory access can lead to significant performance penalties.

**Why Memory Coalescing Matters**
The importance of memory coalescing stems from the nature of modern DRAM (Dynamic Random-Access Memory) used in GPU global memory:
- DRAM Characteristics: DRAM is relatively slow compared to processor speeds. Reading data from DRAM cells involves a time-consuming process of detecting tiny electrical charges.
- Parallelism in Memory Access: To compensate for this slowness, modern DRAMs use parallelism to increase memory access throughput.
- DRAM Bursts: When a DRAM location is accessed, a range of consecutive locations is actually read. This is known as a DRAM burst, which allows for high-speed data transfer once the initial access is made.

**How Memory Coalescing Works**
Memory coalescing takes advantage of the parallel nature of GPU processing:
- Warp Execution: In CUDA architectures, threads within a warp execute the same instruction at any given time.
- Consecutive Access Pattern: The most favorable access pattern occurs when all threads in a warp access consecutive global memory locations.
- Hardware Detection: When threads in a warp execute a load instruction, the hardware detects if they are accessing consecutive memory locations.
- Consolidated Access: If consecutive access is detected, the hardware coalesces these accesses into a single consolidated request for consecutive DRAM locations.

**Benefits of Memory Coalescing**

Memory coalescing offers several advantages:
- Improved Performance: It allows the system to handle multiple memory requests in fewer cycles, significantly enhancing overall performance.
- Efficient Bandwidth Utilization: By consolidating memory accesses, coalescing makes more efficient use of the available memory bandwidth.
- Reduced Latency: Coalesced access patterns can help reduce memory access latency, as fewer separate memory transactions are required.

By understanding and implementing memory coalescing techniques, developers can significantly optimize the performance of parallel computing applications, especially those running on GPU architectures.

**Row-Major vs. Column-Major Order**

For row-major matrices, row-wise access typically yields better performance due to contiguous memory reads, resulting in coalesced memory transactions. Conversely, column-wise access can lead to strided memory patterns, potentially causing uncoalesced memory operations and reduced efficiency. For column-major matrices, the opposite is true: column-wise access provides better coalescing. The key is aligning the access pattern with the storage order to maximize memory bandwidth utilization and minimize cache misses

**Row Major (Column-Wise) Matrix Addition - Non Coalesced Memory Access**

In [24]:
!make SRC=./src/04a_column_wise_add.cu run

nvcc -o ./src/04a_column_wise_add ./src/04a_column_wise_add.cu
././src/04a_column_wise_add
Column-wise matrix addition completed in 8.12403 ms


**Row-Major (Row-wise) Matrix Addition - Coalesced Memory Access**

In [25]:
!make SRC=./src/04b_row_wise_add.cu run

nvcc -o ./src/04b_row_wise_add ./src/04b_row_wise_add.cu
././src/04b_row_wise_add
Row-major matrix addition completed in 7.05098 ms


In [26]:
!make SRC=./src/04a_column_wise_add.cu clean
!make SRC=./src/04b_row_wise_add.cu clean

rm -f ./src/04a_column_wise_add
rm -f ./src/04b_row_wise_add


---
## **05 - Shared Memory Bank Conflict**

Shared memory in NVIDIA GPUs is divided into 32 banks, each 4 bytes wide, designed for parallel access by threads in a warp. Access is efficient if threads target different banks, but bank conflicts occur when multiple threads access the same bank, causing serialized transactions and reduced performance. Avoid conflicts using padding or data alignment.

**Shared Memory Bank Access Types**

<p align="center">
    <img src="./images/no_conflict1.png" alt="Uncoalesced Memory Access" width="30%">
    <img src="./images/no_conflict2.png" alt="Coalesced Memory Access" width="30%">
</p>

From left to right:

1. Linear addressing with a stride of one 32-bit word (no bank conflict).
2. Linear addressing with a stride of two 32-bit words (two-way bank conflict).
3. Linear addressing with a stride of three 32-bit words (no bank conflict).
4. Conflict-free access via random permutation.
5. Conflict-free access since threads 3, 4, 6, 7, and 9 access the same word within bank 5.
6. Conflict-free broadcast access (threads access the same word within a bank).

Bank conflicts occur in shared memory when multiple threads in a warp access different addresses within the same memory bank, except when accessing the same address (broadcast). Linear addressing with a stride of one or three 32-bit words and conflict-free permutations avoid conflicts, while a stride of two words creates two-way conflicts, and broadcasts inherently remain conflict-free.

**Avoiding Bank Conflicts**

<div style="text-align: center;">
  <img src="./images/bank_conflict_padding.png" alt="Avoiding Bank Conflict with Padding">
</div>

- Padding: Adding extra memory to shift indexing and avoid conflicts.
- Alignment: Structuring data to align with bank boundaries.
- Strided Access Management: Ensuring that threads do not access addresses with strides that cause conflicts.
- Conflict-Free Permutations: Reorganizing memory indices for conflict-free access.

**Matrix Transpose with Bank Conflicts**

The line `d_out[y * rows + x] = tile[tid_x][tid_y];` causes a bank conflict when multiple threads in a warp access the same column (`tid_y)` of the shared memory tile (`tile[tid_x][tid_y]`). This happens because shared memory is divided into banks, and simultaneous accesses to the same bank by different threads cause delays.

In [27]:
!make SRC=./src/05a_transpose_conflict.cu run

nvcc -o ./src/05a_transpose_conflict ./src/05a_transpose_conflict.cu
././src/05a_transpose_conflict
Matrix transposition verified successfully!
Execution time: 0.763904 ms


**Matrix Transpose with Bank Padding**

`__shared__ float tile[32][32 + 1];`

 The +1 offsets each row by one element, ensuring that accesses are distributed across different banks, avoiding conflicts, and enabling efficient parallel access.

In [28]:
!make SRC=./src/05b_transpose_padding.cu run

nvcc -o ./src/05b_transpose_padding ./src/05b_transpose_padding.cu
././src/05b_transpose_padding
Matrix transposition verified successfully!
Execution time: 0.208896 ms


In [29]:
!make SRC=./src/05a_transpose_conflict.cu clean
!make SRC=./src/05b_transpose_padding.cu clean

rm -f ./src/05a_transpose_conflict
rm -f ./src/05b_transpose_padding


---
## **06 - Warp Divergence**

<div style="text-align: center;">
  <img src="./images/warp_divergence.bmp" alt="Warp Divergence">
</div>

Warp divergence occurs when threads in a warp (group of 32 threads in CUDA) follow different execution paths due to conditional branches. For example, if some threads in a warp take one branch of an if statement while others take a different branch, the warp must execute both paths sequentially rather than in parallel, causing a performance penalty.

**How is it a Problem?**

- Reduced Parallelism: Ideally, all threads in a warp should execute the same instruction at the same time. With divergence, different threads execute different instructions, which reduces the level of parallelism.
- Performance Penalty: The GPU must serialize execution of divergent branches, executing one path at a time for all threads in the warp, leading to inefficient use of resources and slower performance.

**Solutions to Warp Divergence:**

- Minimize Conditional Branching: Try to avoid if statements that cause divergent paths, especially within the same warp.
- Use Predication: Convert conditional branches into predicated operations where all threads execute the same instructions but only the relevant threads do useful work based on the condition.
- Reorganize Code: Where possible, refactor code so that threads within the same warp are less likely to diverge. For example, you could rearrange data or loop structures to ensure uniform behavior across threads.
- Use Warp-Synchronous Programming: If divergence is unavoidable, try to ensure that divergent branches are as small as possible and isolate them from the rest of the warp execution.

By reducing warp divergence, you improve the parallel efficiency and overall performance of the kernel.

**Code with Warp Divergence**

In [30]:
!make SRC=./src/06a_warp_divergence.cu run

nvcc -o ./src/06a_warp_divergence ./src/06a_warp_divergence.cu
././src/06a_warp_divergence
Time taken with warp divergence: 4.97 ms


Warp divergence occurs because threads within a warp evaluate the condition `if (d_in[idx] % 2 == 0)` differently, causing them to take different execution paths (if or else). CUDA executes these branches sequentially, disabling threads not active in the current branch, which reduces parallel efficiency. This happens because warps execute instructions in lockstep, and divergence forces them to serialize execution. 

**Solving Warp Divergence by Predicating**

In [31]:
!make SRC=./src/06b_warp_predicate.cu run

nvcc -o ./src/06b_warp_predicate ./src/06b_warp_predicate.cu
././src/06b_warp_predicate
Time taken without warp divergence: 4.94 ms


This approach eliminates warp divergence by ensuring all threads in a warp execute the same instruction. Instead of branching with if-else, the ternary operator computes both outcomes and uses the mask to select the correct result for each thread. Since all threads follow the same execution path regardless of the condition, the GPU maintains full warp efficiency without serialization.

In [32]:
!make SRC=./src/06a_warp_divergence.cu clean
!make SRC=./src/06b_warp_predicate.cu clean

rm -f ./src/06a_warp_divergence
rm -f ./src/06b_warp_predicate
